In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TicketCounts").getOrCreate()
# Read the input data from CSV file
df = spark.read.csv("C:/Users/Lenovo/Desktop/shot_logs.csv", header=True)
# Print the schema of the DataFrame
df.printSchema()
#"C:/Users/Lenovo/Desktop/shot_logs.csv"

root
 |-- GAME_ID: string (nullable = true)
 |-- MATCHUP: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- W: string (nullable = true)
 |-- FINAL_MARGIN: string (nullable = true)
 |-- SHOT_NUMBER: string (nullable = true)
 |-- PERIOD: string (nullable = true)
 |-- GAME_CLOCK: string (nullable = true)
 |-- SHOT_CLOCK: string (nullable = true)
 |-- DRIBBLES: string (nullable = true)
 |-- TOUCH_TIME: string (nullable = true)
 |-- SHOT_DIST: string (nullable = true)
 |-- PTS_TYPE: string (nullable = true)
 |-- SHOT_RESULT: string (nullable = true)
 |-- CLOSEST_DEFENDER: string (nullable = true)
 |-- CLOSEST_DEFENDER_PLAYER_ID: string (nullable = true)
 |-- CLOSE_DEF_DIST: string (nullable = true)
 |-- FGM: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_id: string (nullable = true)



# Part1
For each pair of the players (A, B), we define the fear sore of A when facing B is the hit
rate, such that B is closet defender when A is shooting. Based on the fear sore, for each
player, please find out who is his ”most unwanted defender”. (10 pts)

In [7]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Read the shot logs data into a Spark DataFrame
data = spark.read.csv("C:/Users/Lenovo/Desktop/shot_logs.csv", header=True, inferSchema=True)

# Select only the necessary columns
df = data.select("player_name", "SHOT_RESULT", "CLOSEST_DEFENDER")

# Calculate the number of shots taken and number of shots made by each player with each defender
df = df.groupBy("player_name", "CLOSEST_DEFENDER").agg(count(lit(1)).alias("shots_taken"), sum(when(col("SHOT_RESULT") == "made", 1).otherwise(0)).alias("shots_made"))

# Calculate the hit rate for each player and defender pair
df = df.withColumn("hit_rate", col("shots_made") / col("shots_taken"))

# Define a window to partition the data by player name and order by hit rate in ascending order
window = Window.partitionBy("player_name").orderBy(col("hit_rate").asc())

# Find the most unwanted defender for each player
most_unwanted_defender = (
    df.withColumn("rn", row_number().over(window))
      .where(col("rn") == 1)
      .drop("rn")
      .groupBy("player_name")
      .agg(collect_list(col("CLOSEST_DEFENDER")).getItem(0).alias("most_unwanted_defender"))
)

# Show the results
most_unwanted_defender.show(most_unwanted_defender.count())


+--------------------+----------------------+
|         player_name|most_unwanted_defender|
+--------------------+----------------------+
|        aaron brooks|         Nurkic, Jusuf|
|        aaron gordon|        Rivers, Austin|
|     al farouq aminu|        Johnson, James|
|          al horford|           Diaw, Boris|
|        al jefferson|     Hardaway Jr., Tim|
|       alan anderson|            Leuer, Jon|
|         alan crabbe|      Sefolosha, Thabo|
|            alex len|       Knight, Brandon|
|       alexis ajinca|          Meeks, Jodie|
|          alonzo gee|          Korver, Kyle|
|    amare stoudemire|            Deng, Luol|
|        amir johnson|         Grant, Jerami|
|      andre drummond|         James, LeBron|
|      andre iguodala|           Lowry, Kyle|
|        andre miller|          Turner, Evan|
|      andre roberson|        Ginobili, Manu|
|        andrew bogut|          Ibaka, Serge|
|      andrew wiggins|        Sanders, Larry|
|     anthony bennett|        Ajin

# Part2
For each player, we define the comfortable zone of shooting is a matrix of,
{SHOT DIST, CLOSE DEF DIST, SHOT CLOCK}
Please develop a Spark-based algorithm to classify each player’s records into 4 comfortable
zones. Considering the hit rate, which zone is the best for James Harden, Chris Paul,
Stephen Curry, and Lebron James. (10 pts)

In [8]:
from pyspark.sql.functions import col, when, avg

# Load the dataset into a Spark DataFrame
df = spark.read.format("csv").option("header", "true").load("C:/Users/Lenovo/Desktop/shot_logs.csv")

# Calculate the hit rate for each player based on their SHOT_RESULT (made or missed)
player_hits = df.withColumn("HIT_RATE", when(col("SHOT_RESULT") == "made", 1).otherwise(0)) \
    .groupBy("player_name") \
    .agg(avg("HIT_RATE").alias("HIT_RATE"))

# Define 4 comfortable zones based on the hit rate: Very High, High, Medium, and Low
zones = {"Very High": (0.8, 1.0), "High": (0.6, 0.8), "Medium": (0.4, 0.6), "Low": (0.0, 0.4)}

# Assign each record to a comfortable zone based on its hit rate
player_zones = player_hits.withColumn("Comfortable Zone",
                                       when((col("HIT_RATE") >= zones["Very High"][0]) & (col("HIT_RATE") <= zones["Very High"][1]), "Very High")
                                       .when((col("HIT_RATE") >= zones["High"][0]) & (col("HIT_RATE") < zones["High"][1]), "High")
                                       .when((col("HIT_RATE") >= zones["Medium"][0]) & (col("HIT_RATE") < zones["Medium"][1]), "Medium")
                                       .otherwise("Low"))


best_zones = player_zones.groupBy("player_name") \
                         .agg(max("HIT_RATE").alias("max_hit_rate")) \
                         .join(player_zones.withColumnRenamed("HIT_RATE", "max_hit_rate"), on=["player_name", "max_hit_rate"], how="inner") \
                         .select("player_name", "max_hit_rate", "Comfortable Zone")




In [11]:
player_names = ["james harden", "chris paul", "stephen curry", "lebron james"]

for player_name in player_names:
    player_zones = best_zones.filter(best_zones.player_name == player_name)
    player_zones.show()


+------------+------------------+----------------+
| player_name|      max_hit_rate|Comfortable Zone|
+------------+------------------+----------------+
|james harden|0.4497153700189753|          Medium|
+------------+------------------+----------------+

+-----------+-----------------+----------------+
|player_name|     max_hit_rate|Comfortable Zone|
+-----------+-----------------+----------------+
| chris paul|0.480225988700565|          Medium|
+-----------+-----------------+----------------+

+-------------+------------------+----------------+
|  player_name|      max_hit_rate|Comfortable Zone|
+-------------+------------------+----------------+
|stephen curry|0.4855371900826446|          Medium|
+-------------+------------------+----------------+

+------------+------------------+----------------+
| player_name|      max_hit_rate|Comfortable Zone|
+------------+------------------+----------------+
|lebron james|0.4887525562372188|          Medium|
+------------+------------------+

In [10]:
best_zones.show(best_zones.count())

+--------------------+-------------------+----------------+
|         player_name|       max_hit_rate|Comfortable Zone|
+--------------------+-------------------+----------------+
|      andre iguodala| 0.4606413994169096|          Medium|
|   marreese speights| 0.4939759036144578|          Medium|
|       marcin gortat| 0.5456140350877193|          Medium|
|      wesley johnson| 0.4173728813559322|          Medium|
|       trevor booker| 0.4954128440366973|          Medium|
|          mike scott|0.44141689373297005|          Medium|
|        amir johnson| 0.5796019900497512|          Medium|
|           gary neal| 0.3660130718954248|             Low|
|          jeremy lin| 0.4388349514563107|          Medium|
|          dante exum|0.34385964912280703|             Low|
|          trey burke|0.37547892720306514|             Low|
|        jrue holiday| 0.4448818897637795|          Medium|
|        luke babbitt|0.46794871794871795|          Medium|
|michael kidd-gilc...| 0.471544715447154